# Policy Gradient

## 이전 시간

- value based reinforcement learning을 했고, Q라는 action value function을 이용해서 Q function을 구하고 Q function을 통해서 policy를 찾았다.
- DQN도 value based RL이고 DNN으로 Q function을 approximate하고 그것으로 policy를 찾았다.

- state value function, \\(V^\pi(s)\\)
- action value function, \\(Q^\pi(s,a)\\)
- 또한, 앞서서 Monte-carlo나 TD로 문제푸는 것을 배웠다. 그때는 policy에 대한 언급은 없었다.

- table lookup 방법은 더이상 불가능하다. 따라서 value function approximation 방법이 제안되었고, 함수로 value function을 학습하고 policy를 조정한다.

##  Policy를 parameter로 표현할 것이다.

- Model free 방법론 이고,
- value based method는 value function에 기반한 방법론으로 value function을 함수로 표현했고 함수의 parameter를 update해서 그 함수가 정확한 value를 return 하도록 학습했다. 그때 policy는 implicit policy(value function을 이용해서 policy를 만듦)이다.

- policy based, value function을 학습하지 않고, 바로 policy를 배운다.

## 지금 부터는 policy를 output하는 network를 만들 것이다.

- value based
  - Learn value function
  - implicit policy(e-greedy)
- policy based
  - no value function
  - learn policy
- actor-critic
  - learn value function
  - learn policy

## policy gradient 장/단점

- 장점
  - 수렴이 더 잘된다.
  - action이 연속적인 경우에 효과적이다. value based function은 discrete한 output에 대해서 다루었다. action의 차원이 많기 때문에 앞서 value function 방식으로 모든 action을 넣어보고 결정할 수 없다.
  - 확률적인 policy를 배울 수 있다. ex) 가위바위보 게임
- 단점
  - local minima에 빠지기 쉽다.
  - policy의 evaluation이 비효율적이다.
  - variance가 높다.

## value based RL의 문제

- value function을 바탕으로 policy를 계산하기 때문에 value function이 조금만 달라져도 policy에 크게 영향을 받는다. 이 현상은 알고리즘ㅈ 수렴에 불안정을 더한다.
- stochastic policy
  - 때로는 stochastic policy가 최적의 policy가 될 수 있다.

## Objective function

### 어떤 \\(\pi\\)가 더 좋은 \\(\pi\\)인지 정의 되어야한다. \\(\pi\\)가 정의되어야 그 목적을 향해서 update가 된다.

- Q function을 사용하는 action value based 방법에서는 DNN을 update 하기위해서 target과 현재 Q function값의 MSE를 구했다.
  - Q function -> Q function이 나오는 DNN
  - policy -> policy가 나오는 DNN
<br>
- Objective function
  1. state value, 게임에서 보통 똑같은 state에서 시작하기 때문에 처음 시작 state의 value function이 최대로 되게 학습한다.
   - episodic한 환경, 처음 시작 state가 항상 정해져 있고 policy \\(\pi\\)로 게임을 끝까지 했을 때 value를 최대화 하는 문제이다.
  2. average value
   - continuous 환경, Markov chain에서 policy \\(\pi\\)를 따라 움직이다 보면 각 state마다 머무르는 확률을 구할 수 있다.(각 state에 있을 확률 * 그 state에서 value의 모든 총합)
  3. average reward per time-step, 각 time-step 마다 받는 reward를 각 state에서 머무르는 비율(stationary distribution)을 곱한 expectation 사용
   - 한 step 보고 그 state에서 한 step 얻은 기대값
<br>
- 따라서, policy gradient의 목표는 이 objective function을 최대화 시키는 theta라는 policy의 parameter vector를 찾는것이다.
  - how? gradient descent


### 위에서 우리는 objective function \\(J(\theta)\\)를 정의했다.

## Objective function 구하는 법

- Finite difference policy gradient
  - parameter vector가 5개의 차원으로 이루어져 있으면 하나씩 흔들어 보면서 그 gradient를 구한다. policy가 미분가능하지 않아도 사용할 수 있는 방법
- Monte carlo policy gradient
  - objective function에 직접 gradient를 취해준다. policy는 미분 가능하다고 본다.
- actor critic policy gradient

## score function

- \\(\pi\\)는 항상 미분 가능하다고 가정한다.
- Likelihood ratios를 사용해서 계산을 편하게 한다.
- Score function
  - \\(\bigtriangledown_{\theta} \log \pi_{\theta}(s,a) \\)

\\( \begin{split}
\bigtriangledown_{\theta} \pi_{\theta}(s,a) &= \pi_{\theta}(s,a) \frac{\bigtriangledown_{\theta} \pi_{\theta}(s,a)}{\pi_{\theta}(s,a)} \\
&= \pi_{\theta}(s,a) \bigtriangledown_{\theta} \log \pi_{\theta}(s,a) \end{split} \\)

### One-step MDP

- 어떤 분포를 따르는 initial state S에서 시작해서 한 step가고 reward가 끝난다.

\\(\begin{split} J(\theta)
&= \mathbb{E}_{\pi_{\theta}}[r] \\
&= \sum_{s \in \mathcal{S}}d(s) \sum_{a \in \mathcal{A}} \pi_\theta(s,a)R_{s,a}
\end{split}\\)
<br><br>
\\(\begin{split} \bigtriangledown_{\theta} J(\theta)
&= \sum_{s \in \mathcal{S}}d(s) \sum_{a \in \mathcal{A}} \pi_\theta(s,a) \bigtriangledown_{\theta} \log \pi_{\theta}(s,a)R_{s,a} \\
&= \mathbb{E}_{\pi_\theta}[\bigtriangledown_{\theta} \log \pi_{\theta}(s,a)r]
\end{split}\\)

- 위 마지막 식에서 정의된 \\(r\\)은 immediate reward이다.
- \\(\bigtriangledown_{\theta} J(\theta)\\)를 이용해서 policy DNN update한다.
- 그러나 강화학습은 stochastic policy gradient로써 policy 자체가 stochastic하다.
  - policy 2가지 표현, softmax policy, gaussian policy

## Policy gradient theorem

- 앞서 one-step MDP에 대해서 이야기를 했었는데 그렇다면 multi step MDP는?
- Multi step MDP도 likelihood ratio를 일반화 가능하게 하는 것이 policy gradient theorem이다.
- 위의 식 \\(\mathbb{E}_{\pi_\theta}[\bigtriangledown_{\theta} \log \pi_{\theta}(s,a)r]\\)에서 r -> Q로 대체<br><br>
  \\(\bigtriangledown_{\theta} J(\theta) = \mathbb{E}_{\pi_\theta}[\bigtriangledown_{\theta} \log \pi_{\theta}(s,a)Q^{\pi_\theta}(s,a)]\\)

- \\(J(\theta)\\)는 위에서 정의한 3개 다 가능하다.

## Monte carlo policy gradient(REINFORCE)

- episode가 끝나면 해당 policy에 대한 Q function을 구할 수 있다. 하지만 episode가 너무 길면?
- 높은 variance를 가진다.
  - 위 문제를 해결하기 위해서 Neural network를 2개 만들어서 Q function도 approximate해서 gradient를 구하자 -> actor critic policy gradient

- return \\(v_t\\), 축적된 discounted reward, \\( Q^{\pi_\theta}(s_t,a_t)\\)
- \\(\bigtriangleup \theta_t = \alpha \bigtriangledown_{\theta} \log \pi_{\theta}(s_t,a_t)v_t\\)

- 코드
  - \\(\theta\\)를 초기화 하고,
  - \\(\pi\\)에 따라 state, action, reward를 반복하고,
  - \\(\theta \leftarrow \theta + \alpha \bigtriangledown_{\theta} \log \pi_{\theta}(s_t,a_t)v_t\\)
- 결과적으로, 점점 좋은 policy \\(\pi\\)가 만들어진다.

## Actor critic policy gradient

- Monte carlo 방법은 variance가 너무 크기 때문에 critic을 넣는다.

\\( \begin{split} \bigtriangledown_{\theta} J(\theta) &\approx \mathbb{E}_{\pi_\theta}[\bigtriangledown_{\theta} \log \pi_{\theta} (s,a)Q_{w}(s,a)] \\
\bigtriangleup \theta &= \alpha \bigtriangledown_{\theta} \log \pi_{\theta}(s_t,a_t)Q_w(s,a) \end{split}\\)

- 학습 대상이 \\(\theta, w \\)로 두개이다.
- \\(\pi\\)로 \\(\theta\\)를 학습하고, \\(Q\\)로 \\(w\\)를 학습한다.

- Actor, policy approximate하고,
- critic, Q function approximate한다. -> 현재 policy를 평가한다.
  - 따라서 w와 \\(\theta\\) 두개를 사용한다.

#### Q를 학습하는 것은 policy evaluation 문제
- policy evaluation
  - Monte carlo policy evaluation
  - TD learning
  - TD(\\(\lambda \\))

## 코드

- state s와 \\(\theta\\)를 초기화한다.
- \\(\theta\\)에서 action을 하나 찾으면 policy \\(\pi\\)를 이용해서 sample \\(a ~ \pi_\theta \\)
- 매 step 마다 reward알고, 다음 state가 뭔지 안다. 그리고 그 state에서 무슨 action할지 a'를 찾고,<br>
  \\(\delta = r + \gamma Q_w(s', a') - Q_w(s, a)\\), \\(\delta\\)(TD error, w를 update한다), Q(s', a')는 한 step 더 가서 Q, Q(s, a)는 현재 추측<br><br>
- \\(w \leftarrow w + \beta \delta \varnothing(s,a)\\), \\(\beta\\)(learning_rate)<br><br>
- \\(\theta \leftarrow \theta + \alpha \bigtriangledown_{\theta} \log \pi_{\theta}(s_t,a_t)Q_w(s,a)\\)
  - \\(\log \pi_{\theta}(s_t,a_t)\\), 학습 방향을 조정하고,
  - \\(Q_w(s,a)\\), 학습의 조정 크기를 결정한다.
- \\(a \leftarrow a', s \leftarrow s'\\)

## Reducing variance using Baseline

- Actor critic에 baseline을 추가하여 variance 문제를 해결한다.
- 예를 들어) action 1를 했을 때 reward를 100만을 받고, action 2를 했을 때 reward 99만을 받는다면 결국 수렴하게 되겠지만 variance가 너무 크다 따라서 action value function에서 state value function을 빼주어 advantage를 만들어 variance문제를 해결한다.
- baseline을 빼준다.

\\( \begin{split} \mathbb{E}_{\pi_\theta}[\bigtriangledown_{\theta} \log \pi_{\theta} (s,a)B(s)] &= \sum_{s \in \mathcal{S}} d^{\pi_\theta}(s) \sum_a \bigtriangledown_{\theta} \log \pi_{\theta} (s,a)B(s) \;\;\text{- B는 action과 연관이 없기 때문에} \sum \text{ 밖으로 나올 수 있다.} \\
&= \sum_{s \in \mathcal{S}} d^{\pi_\theta}B(s)\bigtriangledown_{\theta} \sum_a \log \pi_{\theta} (s,a) \;\;\text{-}\sum\text{을 다 더하면 1이되고, 미분하면 0이된다.} \\
&= 0
\end{split}\\)

- \\(A^{\pi_\theta}_\theta(s,a) = Q^{\pi_\theta}_w(s,a) - V^{\pi_\theta}_v(s)\\)
- \\(\theta, w, v\\)총 3개의 parameter가 필요하다.

- 하지만 TD error가 advantage function의 smaple이기 때문에<br>
  \\(\delta^{\pi_\theta} = r + \gamma V^{\pi_\theta}(s') - V^{\pi_\theta}(s)\\)

\\( \begin{split}
\mathbb{E}_{\pi_\theta}[\delta^{\pi_\theta}|s, a] &= \mathbb{E}_{\pi_\theta}[r + \gamma V^{\pi_\theta}(s')|s,a] - V^{\pi_\theta}(s) \\
&= Q^{\pi_\theta}(s,a) - V^{\pi_\theta}(s)\\
&= A^{\pi_\theta}(s,a) 
\end{split}\\)
- \\(\delta\\) 1개는 A와 같지 않지만 그들의 평균은 A와 같다.
- 결과적으로,<br>
\\(\bigtriangledown_{\theta} J(\theta) = \mathbb{E}_{\pi_\theta}[\bigtriangledown_{\theta} \log \pi_{\theta} (s,a)\delta^{\pi_\theta}] \\)